In [60]:
import requests

class TokenIssuer:
    _access_token = None

    @classmethod
    def get_token(cls):
        if cls._access_token is None:
            cls.generate_token()
        return cls._access_token

    @classmethod
    def generate_token(cls):
        response = requests.post(
            "https://keycloak.clownchrys.link/realms/default/protocol/openid-connect/token",
            data={
                "client_id": "kubeflow-oidc-authservice",
                "client_secret": "SM7BlsEB0DZW6noQsgcZRw==",
                "username": "admin",
                "password": "thisisauthorized",
                "grant_type": "password"
            }
        ).json()
        cls._access_token = response["access_token"]

# Example 1

In [1]:
# Python 함수를 Component 로 바꿔주는 함수
# decorator 로도 사용할 수 있으며, 여러 옵션을 argument 로 설정할 수 있음
# add_op = create_component_from_func(
#                 func=add,
#                 base_image='python:3.7', # Optional : component 는 k8s pod 로 생성되며, 해당 pod 의 image 를 설정
#                 output_component_file='add.component.yaml', # Optional : component 도 yaml 로 compile 하여 재사용하기 쉽게 관리 가능
#                 packages_to_install=['pandas==0.24'], # Optional : base image 에는 없지만, python code 의 의존성 패키지가 있으면 component 생성 시 추가 가능
#             )
# from kfp.components import create_component_from_func

from kfp import dsl

"""
kfp.components.create_component_from_func :
    Python 함수를 Component 로 바꿔주는 함수
    decorator 로도 사용할 수 있으며, 여러 옵션을 argument 로 설정할 수 있음
    
    add_op = create_component_from_func(
                func=add,
                base_image='python:3.7', # Optional : component 는 k8s pod 로 생성되며, 해당 pod 의 image 를 설정
                output_component_file='add.component.yaml', # Optional : component 도 yaml 로 compile 하여 재사용하기 쉽게 관리 가능
                packages_to_install=['pandas==0.24'], # Optional : base image 에는 없지만, python code 의 의존성 패키지가 있으면 component 생성 시 추가 가능
            )
"""


def add(value_1: int, value_2: int) -> int:
    """
    더하기
    """
    ret = value_1 + value_2
    return ret


def subtract(value_1: int, value_2: int) -> int:
    """
    빼기
    """
    ret = value_1 - value_2
    return ret


def multiply(value_1: int, value_2: int) -> int:
    """
    곱하기
    """
    ret = value_1 * value_2
    return ret


# Python 함수를 선언한 후, kfp.components.create_component_from_func 를 사용하여
# ContainerOp 타입(component)으로 convert
# add_op = create_component_from_func(add)
# subtract_op = create_component_from_func(subtract)
# multiply_op = create_component_from_func(multiply)
add_op = dsl.component(add)
subtract_op = dsl.component(subtract)
multiply_op = dsl.component(multiply)

from kfp.dsl import pipeline


@pipeline(name="add example")
def my_pipeline(value_1: int, value_2: int):
    task_1 = add_op(value_1=value_1, value_2=value_2)
    task_2 = subtract_op(value_1=value_1, value_2=value_2)

    # component 간의 data 를 넘기고 싶다면,
    # output -> input 으로 연결하면 DAG 상에서 연결됨

    # compile 된 pipeline.yaml 의 dag 파트의 dependency 부분 확인
    # uploaded pipeline 의 그래프 확인
    task_3 = multiply_op(value_1=task_1.output, value_2=task_2.output)

/opt/conda/lib/python3.11/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [4]:
import kfp

# kfp dsl compile --py test.py --output pipeline_test.yaml
kfp.compiler.Compiler().compile(my_pipeline, "pipeline_test.yaml")

In [32]:
client = kfp.Client(
    host="http://ml-pipeline.kubeflow.svc.cluster.local:8888",
    verify_ssl=False,
    existing_token=TokenIssuer.get_token()
)
client

In [40]:
run = client.create_run_from_pipeline_func(my_pipeline, arguments={"value_1": 1, "value_2": 2})
run

RunPipelineResult(run_id=cd25fd87-44ad-4c1b-a5d2-9ac820f6b727)

In [ ]:
run.wait_for_run_completion()

# Example 2

In [55]:
import kfp
from kfp import dsl
# from kfp.components import InputPath, OutputPath


# decorator 사용
# @create_component_from_func
@dsl.component
def write_file_op(
    # _path 라는 suffix 를 붙이고, type annotaion 은 OutputPath 로 선언
    data_output_path: dsl.OutputPath(dict)
):
    # package import 문은 함수 내부에 선언
    import json

    # dict data 선언
    data = {
        "a": 300,
        "b": 10,
    }

    # file write to data_output_path
    with open(data_output_path, "w") as f:
        json.dump(data, f)


# @create_component_from_func
@dsl.component
def read_file_and_multiply_op(
    # input 역시, _path 라는 suffix 를 붙이고, type annotation 은 InputPath 로 선언
    data_input_path: dsl.InputPath(dict)
) -> float:
    # package import 문은 함수 내부에 선언
    import json

    # file read to data_output_path
    with open(data_input_path, "r") as f:
        data = json.load(f)

    # multiply
    result = data["a"] * data["b"]

    print(f"Result: {result}")

    return result


@kfp.dsl.pipeline(name="Data Passing by File Example")
def data_passing_file_pipeline():
    write_file_task = write_file_op()
    # _ = read_file_and_multiply_op(write_file_task.outputs["data_output"])
    _ = read_file_and_multiply_op(data_input_path=write_file_task.outputs["data_output_path"])


if __name__ == "__main__":
    kfp.compiler.Compiler().compile(
        data_passing_file_pipeline,
        "./data_passing_file_pipeline.yaml"
    )

In [57]:
client = kfp.Client(
    host="http://ml-pipeline.kubeflow.svc.cluster.local:8888",
    verify_ssl=False,
    existing_token=TokenIssuer.get_token()
)

run = client.create_run_from_pipeline_func(data_passing_file_pipeline, arguments={})
run

RunPipelineResult(run_id=4e4d9ebd-524a-4cec-9e2a-e6a352506784)